In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

import torchio as tio
import torch
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim


from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split


from functions import *

In [2]:
metaData = pd.read_csv('../metaDataTrain.csv')
metaDataClean = metaData.iloc
subject_list = load_training_dataset(metaDataClean)

## Let us construct the features vectors

In [3]:
features = np.zeros((100, 14))  ## there are 5 features and 20 subjects per training class

features_metadata = select_row_x_and_y_from_table(metaDataClean, 2, 3)

## adding into features vectors
features[:,:2] = features_metadata

Let us add volume information to feature vectors

In [4]:
volumes_0_ed = []
volumes_1_ed = []
volumes_2_ed = []
volumes_3_ed = []

volumes_0_es = []
volumes_1_es = []
volumes_2_es = []
volumes_3_es = []
   
for i in range(0, 100):
    subject = subject_list[i]
    
    ed_view = subject.ed_seg
    volumes_0_ed.append(get_volume(ed_view, 0))
    volumes_1_ed.append(get_volume(ed_view, 1))
    volumes_2_ed.append(get_volume(ed_view, 2))
    volumes_3_ed.append(get_volume(ed_view, 3))

    es_view = subject.es_seg
    volumes_0_es.append(get_volume(es_view, 0))
    volumes_1_es.append(get_volume(es_view, 1))
    volumes_2_es.append(get_volume(es_view, 2))
    volumes_3_es.append(get_volume(es_view, 3))

volumes_0_ed = np.array([volumes_0_ed]).flatten()
volumes_1_ed = np.array([volumes_1_ed]).flatten() 
volumes_2_ed = np.array([volumes_2_ed]).flatten() 
volumes_3_ed = np.array([volumes_3_ed]).flatten() 

volumes_0_es = np.array([volumes_0_es]).flatten()
volumes_1_es = np.array([volumes_1_es]).flatten() 
volumes_2_es = np.array([volumes_2_es]).flatten() 
volumes_3_es = np.array([volumes_3_es]).flatten() 

######### new features
ejection_fraction_right = (volumes_1_ed - volumes_1_es) / volumes_1_ed
ejection_fraction_left = (volumes_3_ed - volumes_3_es) / volumes_3_ed

volumes_0_ed = np.array([volumes_0_ed]).flatten()
volumes_1_ed = np.array([volumes_1_ed]).flatten() / volumes_0_ed
volumes_2_ed = np.array([volumes_2_ed]).flatten() / volumes_0_ed
volumes_3_ed = np.array([volumes_3_ed]).flatten() / volumes_0_ed

volumes_0_es = np.array([volumes_0_es]).flatten()
volumes_1_es = np.array([volumes_1_es]).flatten() / volumes_0_es
volumes_2_es = np.array([volumes_2_es]).flatten() / volumes_0_es
volumes_3_es = np.array([volumes_3_es]).flatten() / volumes_0_es

ratioRL_ed = volumes_3_ed / volumes_1_ed
ratioRL_es = volumes_3_es / volumes_1_es

ratioML_ed = volumes_2_ed / volumes_1_ed
ratioML_es = volumes_2_es / volumes_1_es


"""for i, new_col in enumerate([ejection_fraction_right[:], volumes_1_ed[:], volumes_2_ed[:], volumes_3_ed[:], ejection_fraction_left[:], volumes_1_es[:], volumes_2_es[:], volumes_3_es[:]]):
    features[:,i+2] = new_col"""

for i, new_col in enumerate([ratioRL_ed[:], ratioRL_es[:], ratioML_ed[:], ratioML_es[:], ejection_fraction_right[:], volumes_1_ed[:], volumes_2_ed[:], volumes_3_ed[:], ejection_fraction_left[:], volumes_1_es[:], volumes_2_es[:], volumes_3_es[:]]):
    features[:,i+2] = new_col

Let us just give our explicit class vector

In [5]:
classes = np.zeros((100))
classes[:20] = 2
classes[20:40] = 3
classes[40:60] = 1
classes[60:80] = 0
classes[80:100] = 4
classes = np.array(classes.astype('int'))

In [6]:
shapes = np.array([subject.ed.shape for subject in subject_list])
max1, max2, max3 = shapes[:,1].max(), shapes[:,2].max(), shapes[:,3].max()
shaper = tio.CropOrPad((max1, max2, max3))

X_train_ed = np.array([shaper(subject.ed_seg) for subject in subject_list])

C:\Users\Elouan\AppData\Local\Temp\ipykernel_7908\2690441315.py:5: FutureWarning: The input object of type 'ScalarImage' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'ScalarImage', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  X_train_ed = np.array([shaper(subject.ed_seg) for subject in subject_list])
C:\Users\Elouan\AppData\Local\Temp\ipykernel_7908\2690441315.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train_ed = np.array([shaper(subject.ed_seg) for subjec

In [7]:
class encoder(nn.Module):

    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv3d(1, 16, 3, padding=1)
        self.conv2 = nn.Conv3d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv3d(32, 64, 3, padding=1)
        self.conv4 = nn.Conv3d(64, 32, 3, padding=1)
        self.conv5 = nn.Conv3d(32, 16, 3, padding=1)
        self.conv6 = nn.Conv3d(16, 8, 3, padding=1)

        self.maxpool2 = nn.MaxPool3d(2, 2)
        
        self.denseim1 = nn.Linear(int(6656), 128)
        self.denseim2 = nn.Linear(128, 16)

        self.denseclass1 = nn.Linear(30, 12)
        self.denseclass2 = nn.Linear(12, 5)

        """self.batchnorm16 = nn.BatchNorm3d(16)
        self.batchnorm32 = nn.BatchNorm3d(32)
        self.batchnorm64 = nn.BatchNorm3d(64)"""

    def forward(self, im, stats, batchsize):

        image_features = F.relu(self.conv1(im))

        image_features = F.relu(self.conv2(image_features))
        image_features = self.maxpool2(image_features)

        image_features = F.relu(self.conv3(image_features))
        image_features = self.maxpool2(image_features)

        image_features = F.relu(self.conv4(image_features))
        image_features = self.maxpool2(image_features)

        image_features = F.relu(self.conv5(image_features))
        image_features = self.maxpool2(image_features)

        image_features = F.relu(self.conv6(image_features))

        image_features = F.relu(self.denseim1(image_features.view(batchsize, -1)))
        image_features = F.relu(self.denseim2(image_features))

        features = torch.cat((image_features, stats), dim = 1)

        features = F.relu(self.denseclass1(features))

        classification = F.softmax(self.denseclass2(features), dim = 1)

        return classification
    
class customds(Dataset):
    def __init__(self, images, features, targets):
        self.images = images
        self.features = features
        self.targets = targets

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, index):
        image = self.images[index]
        feature = self.features[index]
        target = self.targets[index]
        return image, feature, target

In [8]:
model = encoder()
criterion = torch.nn.CrossEntropyLoss()
lr = 0.5
batchsize = 10
optimizer = torch.optim.Adam(model.parameters(), lr = lr)

In [9]:
images = torch.stack([X_train_ed[i].data for i in range(100)]).float()
classes = torch.from_numpy(classes).float()
classes.requires_grad = True
features = torch.from_numpy(features).float()

dataset = customds(images, features, classes)

dataloader = DataLoader(dataset, batch_size=batchsize, shuffle=True)

In [10]:
def to_device(data, device):

    if isinstance(data, (list, tuple)):
        return [to_device(x, device) for x in data]
        
    return data.to(device, non_blocking = True)

class DeviceDataLoader():

    def __init__(self, dl, device):
        self.dl = dl 
        self.device = device

    def __iter__(self):
        for b in self.dl:
            yield to_device(b, self.device)
    
    def __len__(self):
        return len(self.dl)

def to_device(data, device):
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

dataloader = DeviceDataLoader(dataloader, device)
to_device(model, device)


encoder(
  (conv1): Conv3d(1, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (conv2): Conv3d(16, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (conv3): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (conv4): Conv3d(64, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (conv5): Conv3d(32, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (conv6): Conv3d(16, 8, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (maxpool2): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (denseim1): Linear(in_features=6656, out_features=128, bias=True)
  (denseim2): Linear(in_features=128, out_features=16, bias=True)
  (denseclass1): Linear(in_features=30, out_features=12, bias=True)
  (denseclass2): Linear(in_features=12, out_features=5, bias=True)
)

In [11]:
def model_acc(predict,labels):
  accuracy = torch.sum(predict == labels).item() / len(labels)
  return accuracy

def vector_to_class(x):
  _, y = torch.max(x, dim = 1)
  return y

In [12]:
n_epochs = 3
train_losses=[]
valid_losses=[]
i = 0

for epoch in range(0,n_epochs):
  train_loss=0.0
  
  for batch_images, batch_features, batch_targets in dataloader:

    i+=1
    print('Batch number: {}'.format(i))

    optimizer.zero_grad()
    predict = model(batch_images, batch_features, batchsize)
    # apply loss function
    prediction = vector_to_class(predict).float()
    loss=criterion(prediction,batch_targets)
    # backpropagation
    loss.backward()
    optimizer.step()
    train_loss=loss.item()

  print('Epoch:{} Train Loss:{:.4f}'.format(epoch+1,train_loss/batch_images.shape[0]))
  # calculate accuracy

  print('Accuracy:{:.2f}'.format(model_acc(prediction,batch_targets)))
# END STUDENT CODE

Batch number: 1


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.35 GiB (GPU 0; 4.00 GiB total capacity; 2.50 GiB already allocated; 0 bytes free; 2.52 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF